# Data Storytelling: Food Production and Supply in Africa (2004-2013)

## Introduction

Using the food production and consumption/supply dataset, the aim of this project is to craft a story of the food shortage problem in Africa using appropriate visuals making each variable in the dataset purposeful. This analysis will contain visualisation and narratives so as to show:
* Trends in food production and consumption over the years.
* Average and median food production, the outliers, Quartile and interquartile ranges.
* A comparison between average food consumption and production for each year.

In [ ]:
# Import plotting libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

 ## Food Production

In [ ]:
# opening the production dataset and viewing the first 5 rows

foodprod = pd.read_csv('Africa Food Production.csv')
foodprod.head()

> ### Data Cleaning and Wrangling

In [ ]:
# information on the food production dataset

foodprod.info()

# The dataset contains 23110 rows amd 4 columns
# There are no null or missing values in any of its rows.

In [ ]:
# Checking for the number of unique items produced by all countries

print(foodprod.Item.nunique())

foodprod.Item.unique()

# There are 94 unique items produced by different African countries

In [ ]:
# Descriptive statistics of the food production dataset

foodprod.describe()

> ### Data Exploration and Visualisation

In [ ]:
# grouping by country, year and item to get the total production per item

foodprod_sum = foodprod.groupby(['Country', 'Year', 'Item'])['Value'].sum()
foodprod_sum

In [ ]:
# grouping by country and year to get the total food production per year

foodprod_tot = pd.DataFrame(foodprod.groupby(['Country', 'Year'], as_index = False)['Value'].sum())
foodprod_tot

# Algeria is seen to be the Country with the highest food production per year

In [ ]:
# Sorting by total values in descending order
foodprod_grouped = foodprod_tot.sort_values(by = 'Value', ascending=False)

# Visualising with a line chart the trend  from 2004 - 2013 of total items produced per country
plt.subplots(figsize=(15,10))
lineplt = sns.lineplot(x=foodprod_grouped['Year'], y= foodprod_grouped['Value'], hue=foodprod_grouped['Country'], palette = 'bright', data=foodprod_grouped, ci=None )
lineplt.set_yscale('log')
lineplt.legend(loc='upper right',bbox_to_anchor= (1.2, 1))

# Defining the title of the plot, xaxis and yaxis.
plt.title('Total Food Production per Country from 2004 - 2013')
plt.xlabel('Year', fontsize = 12)
plt.ylabel('Total Food Produced', fontsize = 12);

> It can be observed that Nigeria, Egypt and South Africa are the highest food producing countries between 2004 - 2013.
> It can also be seen that Cabo Verde, Sao Tome and Principe, Djibouti are the lowest food producing countries between 2004 - 2013.

> To effectively observe the change in the food production trend per country between 2004-2013, the data will be visualised using a map. To do this;
>> * A country code dataset will be imported and merged with the food production dataset.
>> * This newly merged dataset will then be used to create a map visualization using the `plotly.express` library. 

In [ ]:
# Importing the country code dataset 

code = pd.read_csv('2014_world_gdp_with_codes.txt')
code.head()

In [ ]:
# Dropping and renaming columns in the country code dataset to fit the column names in the food production dataset

code.columns = ['Country', 'Gdp(billions)','Country Code']

# dropping unwanted column 
code.drop('Gdp(billions)', axis=1, inplace=True)
code.head()

In [ ]:
# Merging the country code dataset with the food production dataset

merged_prod = pd.merge(foodprod_grouped, code, on ='Country', how = 'inner')
merged_prod.head()

In [ ]:
# Creating the map visualisation for the food production of African countries between 2004 and 2013

fig = px.choropleth(merged_prod, locations="Country Code", 
                    color='Value', 
                    hover_name="Country",  
                    animation_frame='Year',
                    scope="africa",
                    color_continuous_scale = px.colors.sequential.Purpor
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

> ### Statistics

In [ ]:
# Creating a boxplot for the food production per country from 2004-2013

fig = go.Figure()
fig.add_trace(go.Box(y = foodprod_grouped.Value,
                     x= foodprod_grouped.Year,
                     boxpoints = "suspectedoutliers",
                     boxmean = True,
                     marker_color='rgb(9,56,125)',
                     line_color='rgb(9,56,125)'
))
fig.update_layout(title_text="Food Production per Country from 2004-2013", yaxis_title= "Total Food Produced", xaxis_title = "Year")
fig.show()

> From the boxplot above, it can be observed that there exists a number of outliers in the food produced for all the years from 2004-2013, with 2012 having the largest outlier. The dataset for the year 2012 will be explored to identify the outlier and other statistics including the mean, median, first quartile, third quartile and interquartile range.

In [ ]:
# Extracting the food production dataset for 2012 to be analysed

foodprod_12 = foodprod.query('Year == 2012')
foodprod_12.shape

In [ ]:
# To find the mean of the food production in 2012

foodprod_12m = foodprod_12.Value.mean()
foodprod_12m

In [ ]:
# To find the median of the food consumption in 2012
foodprod_12med = foodprod_12.Value.median()
foodprod_12med

In [ ]:
# Using numpy to compute the quartile and interquartile range

Q_3, Q_1 = np.percentile(foodprod_12.Value, [75 ,25])
Q_3, Q_1

In [ ]:
# Computing the interquartile range

IQR_ = np.subtract(*np.percentile(foodprod_12.Value, [75 ,25]))
IQR_

In [ ]:
# Computing the upper and lower fence

lower_fence = Q_1 - 1.5*IQR_
upper_fence = Q_3 + 1.5*IQR_
upper_fence, lower_fence

In [ ]:
# Computing the outlier present in the total food consumption in 2012

foodprod_12outliers = foodprod_12[(foodprod_12["Value"] > upper_fence) | (foodprod_12['Value'] < lower_fence)]
foodprod_12outliers

In [ ]:
# Computing the number of countries that present an outlier
foodprod_12outliers.Country.nunique()

> It can be observed that for the food production in African countries in 2012, there are approximately 368 outliers from 38 different countries.

In [ ]:
# Visualizing with a scatterplot to see how the food production varied per country in 2012

fig = px.scatter(foodprod_12, x="Country", y="Value")

# adding horizontal lines to represent the mean, median, upper and lower fence.
fig.update_layout(width=1200,
                 height = 800,
                 autosize=False, shapes=[
                     #add lower fence, mean, median and upper fence lines
                     dict(type= "line", xref= "paper", x0 = 0, x1 = 1, yref = "y", y0 = lower_fence, y1 = lower_fence, line = dict(color = 'Red', width = 1)),
                     dict(type= "line", xref= "paper", x0 = 0, x1 = 1, yref = "y", y0 = foodprod_12m, y1 = foodprod_12m, line = dict(color = 'Orange', width = 1)),
                     dict(type= "line", xref= "paper", x0 = 0, x1 = 1, yref = "y", y0 = foodprod_12med, y1 = foodprod_12med, line = dict(color = 'Blue', width = 1)),
                     dict(type= "line", xref= "paper", x0 = 0, x1 = 1, yref = "y", y0 = upper_fence, y1 = upper_fence, line = dict(color = 'Green', width = 1))
                 ], showlegend=False
                 )

fig.update_traces(textposition = "top center")
fig.update_xaxes(showticklabels = False)
fig.show()

> From the scatterplot above, it can be observed that for 2012, indeed many outliers lie above the upper fence which is represented by the green line. 

## Food Supply

In [ ]:
# Opening and viewing the first 5 rows of the food supply dataset

foodsup = pd.read_csv('Africa Food Supply.csv')
foodsup.head()

> ### Data Wrangling

In [ ]:
# Computing information about the food supply dataset

foodsup.info()

In [ ]:
# Computing the unique number of countries in the food supply dataset

foodsup.Country.nunique()

In [ ]:
# Grouping the supply dataset by country and year to get the total value of items supplied

foodsup_tot = pd.DataFrame(foodsup.groupby(['Country', 'Year'], as_index = False).sum())
foodsup_tot.head()

In [ ]:
# Sorting by total values supplied in descending order
foodsup_grouped = foodsup_tot.sort_values(by='Value', ascending=False)

# Visualising with a line chart the trend  from 2004 - 2013 of total items supplied to each country
plt.subplots(figsize=(15,10))
lineplt1 = sns.lineplot(x=foodsup_grouped['Year'], y= foodsup_grouped['Value'], hue=foodsup_grouped['Country'], palette = 'bright', data=foodsup_grouped, ci=None )
lineplt1.set_yscale('log')
lineplt1.legend(loc='upper right',bbox_to_anchor= (1.2, 1))

# Defining the title of the plot, xaxis and yaxis.
plt.title('Total Supply Value per Year per Country')
plt.xlabel('Year', fontsize = 12)
plt.ylabel('Total Value', fontsize = 12);

> It can be observed that Egypt, Morocco and Tunisia are the countries with the highest food supply between 2004 - 2013.
> It can also be seen that Chad, Madagascar, Zambia are the lowest food producing countries between 2004 - 2013.

> To effectively observe the change in the food supply trend per country between 2004-2013, the data will be visualised using a map. To do this;
> * A country code dataset will be imported and merged with the food supply dataset.
> * This newly merged dataset will then be used to create a map visualization using the `plotly.express` library. 

In [ ]:
# Merging the country code dataset with the food supply dataset

merged_sup = pd.merge(foodsup_grouped, code, on ='Country', how = 'inner')
merged_sup.head()

In [ ]:
# Creating the map visualisation for the food supply in African countries between 2004 and 2013

fig = px.choropleth(merged_sup, locations="Country Code", 
                    color='Value', 
                    hover_name="Country",  
                    animation_frame='Year',
                    scope="africa"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

> ### Statistic

In [ ]:
# Creating a boxplot for the food consumption per country from 2004-2013

fig = go.Figure()
fig.add_trace(go.Box(y = foodsup_grouped.Value,
                     x= foodsup_grouped.Year,
                     boxpoints = "suspectedoutliers",
                     boxmean = True,
                     marker_color='rgb(9,56,125)',
                     line_color='rgb(9,56,125)'
))
fig.update_layout(title_text="Food Consumption per Country from 2004-2013", yaxis_title= "Total Food Consumed", xaxis_title = "Year")
fig.show()

> It can be observed that an outlier exists in the total food consumed for all the years except 2007 and 2013. The dataset for the year 2012 will be explored to identify the outlier and other statistics including the mean, median, first quartile, third quartile and interquartile range.
> As with the production data, 2012 is the year with the largest outlier. Hence the data for 2012 will be explored and the outlier will be analysed

In [ ]:
# Extracting the food consumption dataset for 2012 to be analysed

foodsup_12 = foodsup.query('Year == 2012')

In [ ]:
# To find the mean of the food consumption in 2012

foodsup_12m = foodsup_12.Value.mean()
foodsup_12m

In [ ]:
# To find the median of the food consumption in 2012
foodsup_12med = foodsup_12.Value.median()
foodsup_12med

In [ ]:
# Using numpy to compute the quartile and interquartile range

Q3, Q1 = np.percentile(foodsup_12.Value, [75 ,25])
Q3, Q1

In [ ]:
# Computing the interquartile range

IQR = np.subtract(*np.percentile(foodsup_12.Value, [75 ,25]))
IQR

In [ ]:
# Computing the upper and lower fence

lowerfence = Q1 - 1.5*IQR
upperfence = Q3 + 1.5*IQR
upperfence, lowerfence

In [ ]:
# Computing the outlier present in the total food consumption in 2012

foodsup_12outliers = foodsup_12[(foodsup_12["Value"] > upperfence) | (foodsup_12['Value'] < lowerfence)]
foodsup_12outliers

> The only outlier present in the food consumption for 2012 is Egypt with a total value of  consumed items.

In [ ]:
# Visualizing with a scatterplot to see how the food consumption in Egypt varied from other countries in 2012

fig = px.scatter(foodsup_12, x="Country", y="Value", text ="Country")

# adding horizontal lines to represent the mean, median, upper and lower fence.
fig.update_layout(width=1000,
                 height = 900,
                 autosize=False, shapes=[
                     #add lower fence, mean, median and upper fence lines
                     dict(type= "line", xref= "paper", x0 = 0, x1 = 1, yref = "y", y0 = lowerfence, y1 = lowerfence, line = dict(color = 'Red', width = 1)),
                     dict(type= "line", xref= "paper", x0 = 0, x1 = 1, yref = "y", y0 = foodsup_12m, y1 = foodsup_12m, line = dict(color = 'Orange', width = 1)),
                     dict(type= "line", xref= "paper", x0 = 0, x1 = 1, yref = "y", y0 = foodsup_12med, y1 = foodsup_12med, line = dict(color = 'Blue', width = 1)),
                     dict(type= "line", xref= "paper", x0 = 0, x1 = 1, yref = "y", y0 = upperfence, y1 = upperfence, line = dict(color = 'Green', width = 1))
                 ], showlegend=False
                 )

fig.update_traces(textposition = "top center")
fig.update_xaxes(showticklabels = False)
fig.show()

> From the analysis of both the food production and food supply/consumption data it can be observed that:
Although Nigeria, Egypt and South Africa are the highest food producing countries, of these 3 countries only Egypt is recorded to have one of the highest food supply from 2004-2013.

> Although correlation does not imply causation, it would be expected that the more food a country produces, the more food will be supplied or consumed. For Nigeria and South Africa, this isnt the case as can be observed. 

> ## Rice Production & Population

> The high production of food in some African countries  between 2004 and 2013 may be influenced by its population. Although this is in no way certain, this idea may be measured using hypothesis testing. To understand and test this hypothesis, a particular food common to most countries will be used as a measure. In this case, `Rice (Milled Equivalent)` will be used.

> The dataset containing `Rice (Milled Equivalent)` production for all countries between 2004 and 2013 will be extracted first from the `food production` dataset, then merged with `country code` dataset (contains the country code per country) and the `World Bank population` dataset (contains the measured population per country over a period of years) to create the test data to be used in measuring the proposed hypoyhesis.

For the hypothesis testing the proposed X and Y variables are:
> - X = The amount of rice produced in a year in African countries.
> - Y = The average population of African countries in a year.

The proposed null and alternative hypothesis test statements to be used are given as: 
> Null Hypothesis: There is no significant correlation between the amount of rice produced in a year in African countries and the mean population of African countries in that year. i.e variables X and Y are not correlated.
>> $H_0: U_{corr} =< 0$
>> * where $H_0$ is the notation for null hypothesis.
>> * $U_{corr}$ is the notation for correlation between X and Y.

>  Alternative Hypothesis: There is a significant correlation between the amount of rice produced in a year in African countries and the mean population of African countries in that year. i.e There is a corrrelation between variables X and Y.
>> $H_1: U_{corr} > 0$
>> * where $H_1$ is the notation for alternative hypothesis.
>> * $U_{corr}$ is the notation for correlation between X and Y.

In [ ]:
# Extracting the dataset for rice production in African countries over the years
rice_df = foodprod.query("Item == 'Rice (Milled Equivalent)'")
 
# Finding the sum of values per country per year
rice_tot = pd.DataFrame(rice_df.groupby(['Country', 'Year'], as_index = False)['Value'].sum())
rice_tot

In [ ]:
# Merging the rice dataset with the country code dataset
rice_prod = pd.merge(rice_tot, code, on ='Country', how = 'inner')

# Renaming the Value column in the rice production data
rice_prod.rename(columns = {"Value":"Rice Production (Kilotonnes)"}, inplace=True)
rice_prod.head()

In [ ]:
# Importing the population dataset

pop_data = pd.read_csv("Population data.csv", skiprows = 4)
pop_data.head()

In [ ]:
# Dropping unwanted columns
pop_data.drop(['Indicator Name', 'Indicator Code'], axis = 1, inplace =True)
pop_data.head()

# Extracting only the population recorded between 2004 and 2013
pop_data = pop_data[['Country Name', 'Country Code', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']]
pop_data.head()

In [ ]:
# Cleaning the population dataset using the melt function to convert year rows to columns

pop_df = pd.melt(
    pop_data,
    id_vars=["Country Name", "Country Code"],
    var_name = "Year",
    value_name = "Population"
)
pop_df.head()

In [ ]:
# Checking the datatype for columns in the newly cleaned population dataset
pop_df.dtypes

In [ ]:
# Converting the datatype in the Year column of the population dataset from object to integer
pop_df["Year"] = pop_df["Year"].astype(int)

# Dropping thr country name column
pop_df.drop("Country Name", axis = 1, inplace = True)

In [ ]:
# Merging the population and rice production dataset using the country code and year

pop_riceprod = pd.merge(rice_prod, pop_df, on= ["Country Code", "Year"], how = "inner")
pop_riceprod.head()

In [ ]:
# Computing the average rice production and popopulation per year

test_data = pd.DataFrame(pop_riceprod.groupby("Year", as_index=False).mean())
test_data

In [ ]:
# Performing the Pearson Correlation Test to measure the correlation between the average rice production and population

from scipy import stats
x = np.array(test_data['Rice Production (Kilotonnes)'])
y = np.array(test_data['Population'])

stats.pearsonr(x, y)

> The results above show a high  positive correlation of 0.97and a p-value of 4.99 x 10e-6. This means that there’s a very strong correlation between X and Y, and such a small p-value suggests that the null hypothesis ($H_0$) is unlikely to be true. Therefore we reject the null hypothesis at the 5% level based on the Pearson Correlation test.

> It can therefore be concluded that the correlation coefficient (0.97) is significantly far from 0 i.e., there is a significant correlation between mean rice production in a year and the mean population of African countries for that year.

> It is also very important to state that correlation does not imply causation. Although there exists a strong correlation between population and rice production, this doesn’t necessarily mean that the production of rice drives population, or vice versa.

In [ ]:
# visualising the test data with a scatter plot

fig = px.scatter(test_data, x = "Rice Production (Kilotonnes)", y = "Population", text = "Year", title="Population vs Rice Production (2004 - 2013)")
fig.show();

> The scatterplot above shows a positive corrrelation between rice production and the population in African countries over the years. As the population increases, the average amount of rice produced tends to increase.

In [ ]:
# Creating a map plot of Africa showing how the production of rice varies per country over the years

fig = px.choropleth(pop_riceprod, locations="Country Code", 
                    color='Rice Production (Kilotonnes)', 
                    hover_name="Country",  
                    animation_frame='Year',
                    scope="africa"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# To obtain the top countries with the highest rice production in kilotonnes over the years

top = pd.DataFrame(pop_riceprod.groupby(['Country', 'Year'], as_index = False)['Rice Production (Kilotonnes)'].sum())
top.sort_values(by = 'Rice Production (Kilotonnes)', ascending= False, inplace =True)
top

In [ ]:
# Visualising in a plot the top countries with the highest rice production in kilotonnes over the years

plt.subplots(figsize=(15,10))
lineplt = sns.lineplot(x=top['Year'], y= top['Rice Production (Kilotonnes)'], hue=top['Country'], palette = 'bright', data=top, ci=None )
lineplt.set_yscale('log')
lineplt.legend(loc='upper right',bbox_to_anchor= (1.2, 1))

plt.title('Average Rice Production in Kilotonnes  per Country')
plt.xlabel('Country', fontsize = 12)
plt.ylabel('Average Rice Production', fontsize = 12);

> It can be observed that `Egypt`, `Nigeria` and `Madagascar` are the highest `Rice (Milled Equivalent)` producing countries between 2004 and 2013.
> To compare if these 3 countries have any other products in common i.e Egypt, Nigeria and Madagascar, a dataframe will be created for each country with their total produce. Then the items in each of these dataframes will be compared to see what they have in common.

In [ ]:
# From the original production dataset, 3 dataframes for Egypt, Nigeria and Madagascar are extracted

Egypt = foodprod.query('Country == "Egypt"')
Nigeria = foodprod.query('Country == "Nigeria"')
Madagascar = foodprod.query('Country == "Madagascar"')

In [ ]:
# Comparing between Egypt and Nigeria to find other simmilar produced items between them

Egy_Nig = Egypt['Item'][Egypt['Item'].isin(Nigeria['Item'])].unique()
Egy_Nig

In [ ]:
# To compute the total number of similar items produced by both Egypt and Nigeria

len(Egy_Nig)

In [ ]:
# Comparing between Egypt and Madagascar to find other simmilar produced items between them

Egy_Mad= Egypt['Item'][Egypt['Item'].isin(Madagascar['Item'])].unique()
Egy_Mad

In [ ]:
# To compute the total number of similar items produced by both Egypt and Madagascar

len(Egy_Mad)

In [ ]:
# Comparing between Nigeria and Madagascar to find other simmilar produced items between them

Nig_Mad= Nigeria['Item'][Nigeria['Item'].isin(Madagascar['Item'])].unique()
Nig_Mad

In [ ]:
# To compute the total number of similar items produced by both Nigeria and Madagascar

len(Nig_Mad)

### Food Supply & Population

> The high production of food in some African countries does have a strong correlation with the population. A similar hypothesis will be tested on the food supply dataset to measure if the food supply in African countries is correlated with the average population of that country.

> The merged supply dataset `merged_sup` containing the food supply and `country code` dataset (contains the country code per country) will be merged with the cleaned population dataset `pop_df` to create the test data to be used in measuring the proposed hypoyhesis.

For the hypothesis testing the proposed X and Y variables are:
> - X = The amount of food supplied in a year in African countries.
> - Y = The average population of African countries in a year.

The proposed null and alternative hypothesis test statements to be used are given as: 
> Null Hypothesis: There is no significant correlation between the amount of food supplied in a year in African countries and the mean population of African countries in that year. i.e variables X and Y are not correlated.
>> $H_0: U_{corr} =< 0$
>> * where $H_0$ is the notation for null hypothesis.
>> * $U_{corr}$ is the notation for correlation between X and Y.

>  Alternative Hypothesis: There is a significant correlation between the amount of food supplied in a year in African countries and the mean population of African countries in that year. i.e There is a corrrelation between variables X and Y.
>> $H_1: U_{corr} > 0$
>> * where $H_1$ is the notation for alternative hypothesis.
>> * $U_{corr}$ is the notation for correlation between X and Y.

In [ ]:
# Merging the merged_sup dataset with the country code dataset
pop_sup = pd.merge(merged_sup, pop_df, on= ["Country Code", "Year"], how = "inner")

# Renaming the Value column in the newly merged data
pop_sup.rename(columns = {"Value":"Food Supplied"}, inplace=True)
pop_sup.head()

In [ ]:
# Computing the average food supply and popopulation per year

test_datas = pd.DataFrame(pop_sup.groupby("Year", as_index=False).mean())
test_datas

In [ ]:
# Performing the Pearson Correlation Test to measure the correlation between the average food supply and population

from scipy import stats
x = np.array(test_datas['Food Supplied'])
y = np.array(test_datas['Population'])

stats.pearsonr(x, y)

> The results above show a high positive correlation of 0.99 and a p-value of 3.38 x 10e-8. This means that there’s a very strong correlation between X and Y, and such a small p-value suggests that the null hypothesis ($H_0$) is unlikely to be true. Therefore we reject the null hypothesis at the 5% level based on the Pearson Correlation test.

> It can therefore be concluded that the correlation coefficient (0.99) is significantly far from 0 i.e., there is a significant correlation between mean food supplied in a year and the mean population of African countries for that year.

> It is also very important to state that correlation does not imply causation. Although there exists a strong correlation between population and food supply, this doesn’t necessarily mean that the population drives food supply, or vice versa.

In [ ]:
# visualising the test_datas with a scatter plot

fig = px.scatter(test_datas, x = "Food Supplied", y = "Population", text = "Year", title="Population vs Food Supply (2004 - 2013)")
fig.show();

> The scatterplot above shows a positive corrrelation between food supplied and the population in African countries over the years. As the population increases, the average amount of food supplied tends to increase.

## Conclusion

> From the analysis carried out above for the food produced and food supplied in African Countries between 2004 - 2013, it can be concluded that:
> * There is a strong positive correlation between the average production of rice and the population between 2004 and 2013. 
> * There is also a strong positive correlation between the average food supplied and the population between 2004 and 2013.

Although, this does not imply that an increase in population drives an increase in rice production, or that an increase in population drives an increase in food supply, it establishes the relationship between the two variables.
> * The countries with the highest food production dont have high food supply i.e the amount of food supplied over the years is relatively smaller compared to the amount of food being produced.
> * Although there is a high amount of food production and population over the years, there is a low amount smaller food supply when compared to the amount of food produced. It can be deduced that there is a possible food shortage in African countries between 2004 and 2013.
> * Based on the data analysed and hypothesis tested, the population does have a strong correlated relationship with both the food production and food supply in Africa between 2004 & 2013. Fom the hypothesis result, population doesnt seem to explain the possible cause of the food shortage in Africa during this time. Although other factors may also contribute to the food shortage like price inflation, exportation or poor storage facilities etc this cannot be confirmed until extra data is available to analyse these possibilities. 
